# 1. Creating the SGraph

#### Implementation of the basic SGraph class that contains the label information of the 2-regular digraph and the ON / OFF switches on the vertices. Creating of a reset function to reset all switches to OFF states.

In [1]:
class Switch:
    def __init__(self, state=False):
        self.state = state

    def toggle(self):
        self.state = not self.state

    def is_on(self):
        return self.state


class Robot:
    def __init__(self, vertex):
        self.current_vertex = vertex

    def move(self, edge_label):
        if edge_label == 'a':
            self.current_vertex = self.current_vertex.outgoing_edges['a']
        elif edge_label == 'b':
            self.current_vertex = self.current_vertex.outgoing_edges['b']
        self.current_vertex.switch.toggle()


class Vertex:
    def __init__(self, name):
        self.name = name
        self.outgoing_edges = {'a': None, 'b': None}
        self.switch = Switch()

    def add_outgoing_edge(self, edge_label, destination_vertex):
        self.outgoing_edges[edge_label] = destination_vertex


class SGraph:
    def __init__(self, graph_string):
        self.vertices = {}
        self.robot = None
        
        for vertex_name in graph_string.split():
            vertex = Vertex(vertex_name)
            self.vertices[vertex_name] = vertex

        vertex_names = list(self.vertices.keys())
        for i, vertex_name in enumerate(vertex_names):
            vertex = self.vertices[vertex_name]
            vertex.add_outgoing_edge('a', self.vertices[vertex_names[(i + 1) % len(vertex_names)]])
            vertex.add_outgoing_edge('b', self.vertices[vertex_names[(i + 2) % len(vertex_names)]])

    def set_robot_position(self, vertex_name):
        if vertex_name in self.vertices:
            self.robot = Robot(self.vertices[vertex_name])
        else:
            raise ValueError("Invalid vertex name")

    def reset_switches(self):
        for vertex in self.vertices.values():
            vertex.switch.state = False
            
    def display_graph(self):
        print("===== Graph State =====")
        for vertex_name, vertex in self.vertices.items():
            print(f"Vertex {vertex_name} (Switch: {'ON' if vertex.switch.is_on() else 'OFF'})")
            print("Outgoing Edges:")
            for edge_label, destination_vertex in vertex.outgoing_edges.items():
                print(f"Edge {edge_label} -> Vertex {destination_vertex.name}")
        print("=======================")

# Implementation of SGraph 

In [2]:
graph_string = "A B C D E F"
graph = SGraph(graph_string)
graph.set_robot_position('A')
graph.robot.move('a')
graph.robot.move('b')
graph.robot.move('a')
graph.display_graph()

#Resetting
graph.reset_switches()
graph.display_graph()

===== Graph State =====
Vertex A (Switch: OFF)
Outgoing Edges:
Edge a -> Vertex B
Edge b -> Vertex C
Vertex B (Switch: ON)
Outgoing Edges:
Edge a -> Vertex C
Edge b -> Vertex D
Vertex C (Switch: OFF)
Outgoing Edges:
Edge a -> Vertex D
Edge b -> Vertex E
Vertex D (Switch: ON)
Outgoing Edges:
Edge a -> Vertex E
Edge b -> Vertex F
Vertex E (Switch: ON)
Outgoing Edges:
Edge a -> Vertex F
Edge b -> Vertex A
Vertex F (Switch: OFF)
Outgoing Edges:
Edge a -> Vertex A
Edge b -> Vertex B
===== Graph State =====
Vertex A (Switch: OFF)
Outgoing Edges:
Edge a -> Vertex B
Edge b -> Vertex C
Vertex B (Switch: OFF)
Outgoing Edges:
Edge a -> Vertex C
Edge b -> Vertex D
Vertex C (Switch: OFF)
Outgoing Edges:
Edge a -> Vertex D
Edge b -> Vertex E
Vertex D (Switch: OFF)
Outgoing Edges:
Edge a -> Vertex E
Edge b -> Vertex F
Vertex E (Switch: OFF)
Outgoing Edges:
Edge a -> Vertex F
Edge b -> Vertex A
Vertex F (Switch: OFF)
Outgoing Edges:
Edge a -> Vertex A
Edge b -> Vertex B


# 2. (a) Enumerator Class

#### Creating an efficient Enumerator for all 2-regular digraph with different labels.

In [24]:
from itertools import product

class GraphEnumerator:
    def __init__(self, m):
        self.m = m
        self.switch_states = []
        self.edge_labels = []
        self.current_index = 0
        
        self.generate_switch_states()
        self.generate_edge_labels()
        
    def generate_switch_states(self):
        self.switch_states = [format(i, f'0{self.m}b') for i in range(2**self.m)]
        
    def generate_edge_labels(self):
        self.edge_labels = [''.join(label) for label in product('ab', repeat=self.m)]
        
    def SetCurrent(self, current):
        self.current_index = current
        
    def Next(self):
        if self.current_index < len(self.switch_states) * len(self.edge_labels):
            switch_index = self.current_index // len(self.edge_labels)
            edge_index = self.current_index % len(self.edge_labels)
            
            current_switch_states = self.switch_states[switch_index]
            current_edge_labels = self.edge_labels[edge_index]
            
            self.current_index += 1
            return current_switch_states, current_edge_labels
        else:
            return None, None

def construct_graph(switch_states, edge_labels):
    m = len(switch_states)
    graph = {i: {"switch": switch_states[i], "edges": {"a": edge_labels[2*i], "b": edge_labels[2*i + 1]}} for i in range(m)}
    return graph

def move_robot(graph, start_vertex):
    current_vertex = start_vertex
    visited = set()
    
    while current_vertex not in visited:
        visited.add(current_vertex)
        switch_state = graph[current_vertex]["switch"]
        print(f"At vertex {current_vertex} with switch state {switch_state}")     
        edge_label = graph[current_vertex]["edges"][switch_state]
        next_vertex = int(edge_label, 2)
        graph[next_vertex]["switch"] = "ON" if graph[next_vertex]["switch"] == "OFF" else "OFF"        
        current_vertex = next_vertex

# Implementation of Enumerator

In [28]:
m = 4
enumerator = GraphEnumerator(m)

while True:
    switch_states, edge_labels = enumerator.Next()
    if switch_states is None:
        break
    print(f"Processing switch states: {switch_states}, edge labels: {edge_labels}")    
    if len(edge_labels) != 2 * m:
        continue 
    
    graph = construct_graph(switch_states, edge_labels)
    print(f"Switch states: {switch_states}, Edge labels: {edge_labels}")
    
    start_vertex = 0  
    move_robot(graph, start_vertex)
    print("Robot movement completed\n")

Processing switch states: 0000, edge labels: aaaa
Processing switch states: 0000, edge labels: aaab
Processing switch states: 0000, edge labels: aaba
Processing switch states: 0000, edge labels: aabb
Processing switch states: 0000, edge labels: abaa
Processing switch states: 0000, edge labels: abab
Processing switch states: 0000, edge labels: abba
Processing switch states: 0000, edge labels: abbb
Processing switch states: 0000, edge labels: baaa
Processing switch states: 0000, edge labels: baab
Processing switch states: 0000, edge labels: baba
Processing switch states: 0000, edge labels: babb
Processing switch states: 0000, edge labels: bbaa
Processing switch states: 0000, edge labels: bbab
Processing switch states: 0000, edge labels: bbba
Processing switch states: 0000, edge labels: bbbb
Processing switch states: 0001, edge labels: aaaa
Processing switch states: 0001, edge labels: aaab
Processing switch states: 0001, edge labels: aaba
Processing switch states: 0001, edge labels: aabb


# 3. Addition of function MOVE in SGraph

#### The initpos is the starting vertex of the robot. The input is the string of sequential edge labels. After executing this function the SGraph object contains the correct outcome of the movement i.e. the states of the switches. The return value is the final position of the robot.

In [5]:
class Robot:
    def __init__(self, vertex):
        self.current_vertex = vertex

    def move(self, edge_label):
        if edge_label == 'a':
            self.current_vertex = self.current_vertex.outgoing_edges['a']
        elif edge_label == 'b':
            self.current_vertex = self.current_vertex.outgoing_edges['b']
        self.current_vertex.switch.toggle()

class SGraph:
    def __init__(self, graph_string):
        self.vertices = {}
        self.robot = None
        for vertex_name in graph_string.split():
            vertex = Vertex(vertex_name)
            self.vertices[vertex_name] = vertex
        vertex_names = list(self.vertices.keys())
        for i, vertex_name in enumerate(vertex_names):
            vertex = self.vertices[vertex_name]
            vertex.add_outgoing_edge('a', self.vertices[vertex_names[(i + 1) % len(vertex_names)]])
            vertex.add_outgoing_edge('b', self.vertices[vertex_names[(i + 2) % len(vertex_names)]])

    def set_robot_position(self, vertex_name):
        if vertex_name in self.vertices:
            self.robot = Robot(self.vertices[vertex_name])
        else:
            raise ValueError("Invalid vertex name")

    def reset_switches(self):
        for vertex in self.vertices.values():
            vertex.switch.state = False
            
    def display_graph(self):
        print("===== Graph State =====")
        for vertex_name, vertex in self.vertices.items():
            print(f"Vertex {vertex_name} (Switch: {'ON' if vertex.switch.is_on() else 'OFF'})")
            print("Outgoing Edges:")
            for edge_label, destination_vertex in vertex.outgoing_edges.items():
                print(f"Edge {edge_label} -> Vertex {destination_vertex.name}")
        print("=======================")
        
    def Move(self, initpos, input):
        if initpos not in self.vertices:
            raise ValueError("Invalid initial position")
        self.set_robot_position(initpos)
        for edge_label in input:
            self.robot.move(edge_label)
        final_position = self.robot.current_vertex.name
        return final_position

# Implementation of the function Move

In [6]:
graph_string = "A B C D"
graph = SGraph(graph_string)
init_position = "A"
input_sequence = "abbaab"
final_position = graph.Move(init_position, input_sequence)
graph.display_graph()
print("Final Position:", final_position)

===== Graph State =====
Vertex A (Switch: OFF)
Outgoing Edges:
Edge a -> Vertex B
Edge b -> Vertex C
Vertex B (Switch: ON)
Outgoing Edges:
Edge a -> Vertex C
Edge b -> Vertex D
Vertex C (Switch: ON)
Outgoing Edges:
Edge a -> Vertex D
Edge b -> Vertex A
Vertex D (Switch: OFF)
Outgoing Edges:
Edge a -> Vertex A
Edge b -> Vertex B
Final Position: B


# 4. (a) Addition of the function COUNT IN SGraph

#### Implement a public function Count(initpos, dest, n) in SGraph class. The initpos is the starting vertex of the robot. The 'dest' is the destination vertex. Counting the number of different commands of length n that move the robot from initpos to dest. The return value is that number.

In [7]:
class Robot:
    def __init__(self, vertex):
        self.current_vertex = vertex

    def move(self, edge_label):
        if edge_label == 'a':
            self.current_vertex = self.current_vertex.outgoing_edges['a']
        elif edge_label == 'b':
            self.current_vertex = self.current_vertex.outgoing_edges['b']
        self.current_vertex.switch.toggle()

    def count_commands(self, current_vertex, destination_vertex, n):
        if n == 0:
            return 1 if current_vertex == destination_vertex else 0
        count = 0
        for edge_label in current_vertex.outgoing_edges:
            next_vertex = current_vertex.outgoing_edges[edge_label]
            count += self.count_commands(next_vertex, destination_vertex, n - 1)
        return count

class SGraph:
   
    def __init__(self, graph_string):
        self.vertices = {}
        self.robot = None
        for vertex_name in graph_string.split():
            vertex = Vertex(vertex_name)
            self.vertices[vertex_name] = vertex
        vertex_names = list(self.vertices.keys())
        for i, vertex_name in enumerate(vertex_names):
            vertex = self.vertices[vertex_name]
            vertex.add_outgoing_edge('a', self.vertices[vertex_names[(i + 1) % len(vertex_names)]])
            vertex.add_outgoing_edge('b', self.vertices[vertex_names[(i + 2) % len(vertex_names)]])

    def set_robot_position(self, vertex_name):
        if vertex_name in self.vertices:
            self.robot = Robot(self.vertices[vertex_name])
        else:
            raise ValueError("Invalid vertex name")

    def reset_switches(self):
        for vertex in self.vertices.values():
            vertex.switch.state = False
            
    def display_graph(self):
        print("===== Graph State =====")
        for vertex_name, vertex in self.vertices.items():
            print(f"Vertex {vertex_name} (Switch: {'ON' if vertex.switch.is_on() else 'OFF'})")
            print("Outgoing Edges:")
            for edge_label, destination_vertex in vertex.outgoing_edges.items():
                print(f"Edge {edge_label} -> Vertex {destination_vertex.name}")
        print("=======================")

    def Count(self, initpos, dest, n):
        if initpos not in self.vertices or dest not in self.vertices:
            raise ValueError("Invalid initial or destination vertex")
        self.set_robot_position(initpos)
        count = self.robot.count_commands(self.robot.current_vertex, self.vertices[dest], n)
        return count

# Implementation of 'COUNT' in the SGraph

In [8]:
graph_string = "A B C D"
graph = SGraph(graph_string)
init_position = "A"
destination_position = "D"
n = 5
number_of_commands = graph.Count(init_position, destination_position, n)
print("Number of different commands:", number_of_commands)

Number of different commands: 10


# 5. Addition of the function 'SOLVE'  to SGraph

#### Implement a public function Solve(initpos, states) in SGraph class. The initpos is the starting vertex of the robot. The states is the 0/1 array of the interested ON/OFF states of the switches. This function returns one of the shortest commands that move the robot to toggle the switches to match the states.

In [9]:
from collections import deque

class Robot:
    def __init__(self, vertex):
        self.current_vertex = vertex

    def move(self, edge_label):
        if edge_label == 'a':
            self.current_vertex = self.current_vertex.outgoing_edges['a']
        elif edge_label == 'b':
            self.current_vertex = self.current_vertex.outgoing_edges['b']
        self.current_vertex.switch.toggle()

    def count_commands(self, current_vertex, destination_vertex, n):
        if n == 0:
            return 1 if current_vertex == destination_vertex else 0

        count = 0
        for edge_label in current_vertex.outgoing_edges:
            next_vertex = current_vertex.outgoing_edges[edge_label]
            count += self.count_commands(next_vertex, destination_vertex, n - 1)

        return count

class SGraph:
    def __init__(self, graph_string):
        self.vertices = {}
        self.robot = None

        for vertex_name in graph_string.split():
            vertex = Vertex(vertex_name)
            self.vertices[vertex_name] = vertex

        vertex_names = list(self.vertices.keys())
        for i, vertex_name in enumerate(vertex_names):
            vertex = self.vertices[vertex_name]
            vertex.add_outgoing_edge('a', self.vertices[vertex_names[(i + 1) % len(vertex_names)]])
            vertex.add_outgoing_edge('b', self.vertices[vertex_names[(i + 2) % len(vertex_names)]])

    def set_robot_position(self, vertex_name):
        if vertex_name in self.vertices:
            self.robot = Robot(self.vertices[vertex_name])
        else:
            raise ValueError("Invalid vertex name")

    def reset_switches(self):
        for vertex in self.vertices.values():
            vertex.switch.state = False
            
    def display_graph(self):
        print("===== Graph State =====")
        for vertex_name, vertex in self.vertices.items():
            print(f"Vertex {vertex_name} (Switch: {'ON' if vertex.switch.is_on() else 'OFF'})")
            print("Outgoing Edges:")
            for edge_label, destination_vertex in vertex.outgoing_edges.items():
                print(f"Edge {edge_label} -> Vertex {destination_vertex.name}")
        print("=======================")

    def Count(self, initpos, dest, n):
        if initpos not in self.vertices or dest not in self.vertices:
            raise ValueError("Invalid initial or destination vertex")
        self.set_robot_position(initpos)
        count = self.robot.count_commands(self.robot.current_vertex, self.vertices[dest], n)
        return count


    def Solve(self, initpos, states):
        if initpos not in self.vertices:
            raise ValueError("Invalid initial position")
        self.set_robot_position(initpos)
        target_states = tuple(states)
        queue = deque([(self.robot.current_vertex, [])])
        visited = set()

        while queue:
            current_vertex, command = queue.popleft()
            current_states = tuple(vertex.switch.is_on() for vertex in self.vertices.values())
            if current_states == target_states:
                return "".join(command) 
            if (current_vertex, current_states) in visited:
                continue
            visited.add((current_vertex, current_states))

            for edge_label, next_vertex in current_vertex.outgoing_edges.items():
                next_command = command.copy()
                next_command.append(edge_label)
                next_states = current_states[:]
                next_vertex.switch.toggle()
                next_states = tuple(vertex.switch.is_on() for vertex in self.vertices.values())
                queue.append((next_vertex, next_command))
                
        return "No solution found"  

# Implementation of Solve in SGraph

In [10]:
graph_string = "A B C"
graph = SGraph(graph_string)

init_position = "A"
desired_states = [True, False, True]  

solution = graph.Solve(init_position, desired_states)
print("Solution:", solution)


Solution: ba


# BONUS QUESTIONS

## 4. (b) Creation of function SCount

In [11]:
class Robot:
    def __init__(self, current_vertex):
        self.current_vertex = current_vertex
        self.previous_vertex = None
        self.visited_vertices = []

    def move(self, edge_label):
        destination_vertex = self.current_vertex.outgoing_edges.get(edge_label)
        if destination_vertex:
            self.visited_vertices.append(self.current_vertex)
            self.previous_vertex = self.current_vertex
            self.current_vertex = destination_vertex
        else:
            raise ValueError("Invalid edge label")

    def move_back(self):
        if self.previous_vertex:
            self.current_vertex, self.previous_vertex = self.previous_vertex, self.current_vertex
        else:
            raise ValueError("Cannot move back from the starting position")

class Vertex:
    def __init__(self, name):
        self.name = name
        self.outgoing_edges = {}
        self.switch = Switch(name)

    def add_outgoing_edge(self, label, destination_vertex):
        self.outgoing_edges[label] = destination_vertex

class Switch:
    def __init__(self, name):
        self.name = name
        self.state = False

    def is_on(self):
        return self.state

    def states_match(self, target_states):
        return self.state == target_states[self.name]



In [12]:
class SGraph:
   
    def __init__(self, graph_string):
        self.vertices = {}
        self.robot = None
        for vertex_name in graph_string.split():
            vertex = Vertex(vertex_name)
            self.vertices[vertex_name] = vertex
        vertex_names = list(self.vertices.keys())
        for i, vertex_name in enumerate(vertex_names):
            vertex = self.vertices[vertex_name]
            vertex.add_outgoing_edge('a', self.vertices[vertex_names[(i + 1) % len(vertex_names)]])
            vertex.add_outgoing_edge('b', self.vertices[vertex_names[(i + 2) % len(vertex_names)]])

    def set_robot_position(self, vertex_name):
        if vertex_name in self.vertices:
            self.robot = Robot(self.vertices[vertex_name])
        else:
            raise ValueError("Invalid vertex name")

    def reset_switches(self):
        for vertex in self.vertices.values():
            vertex.switch.state = False
            
    def display_graph(self):
        print("===== Graph State =====")
        for vertex_name, vertex in self.vertices.items():
            print(f"Vertex {vertex_name} (Switch: {'ON' if vertex.switch.is_on() else 'OFF'})")
            print("Outgoing Edges:")
            for edge_label, destination_vertex in vertex.outgoing_edges.items():
                print(f"Edge {edge_label} -> Vertex {destination_vertex.name}")
        print("=======================")

    def Count(self, initpos, dest, n):
        if initpos not in self.vertices or dest not in self.vertices:
            raise ValueError("Invalid initial or destination vertex")
        self.set_robot_position(initpos)
        count = self.robot.count_commands(self.robot.current_vertex, self.vertices[dest], n)
        return count

    def SCount(self, initpos, dest, n, states):
        def dfs_count_commands(current_vertex, current_length, target_length, target_vertex, target_states):
            if current_length == target_length:
                if current_vertex == target_vertex and current_vertex.switch.states_match(target_states):
                    return 1
                return 0

            count = 0
            for edge_label, destination_vertex in current_vertex.outgoing_edges.items():
                self.robot.move(edge_label)
                count += dfs_count_commands(destination_vertex, current_length + 1, target_length, target_vertex, target_states)
                self.robot.move_back()

            return count

        if initpos not in self.vertices or dest not in self.vertices:
            raise ValueError("Invalid vertex name")

        if n < 0:
            raise ValueError("n must be a non-negative integer")

        if not isinstance(states, dict):
            raise ValueError("The states must be a dictionary with vertex names as keys")

        self.set_robot_position(initpos)
        count = dfs_count_commands(self.robot.current_vertex, 0, n, self.vertices[dest], states)
        return count

## Implementation of Scount function

In [13]:
graph_string = "a b c d g h"
graph = SGraph(graph_string)
states = {
    "a": 1,
    "b": 0,
    "c": 1,
    "d": 0,
    "g": 1,
    "h": 0,
}
initpos = "a"
graph.set_robot_position(initpos)
dest = "d"
n = 3
count = graph.SCount(initpos, dest, n, states)
print(f"Number of valid commands from {initpos} to {dest} with switch states {states}: {count}")

Number of valid commands from a to d with switch states {'a': 1, 'b': 0, 'c': 1, 'd': 0, 'g': 1, 'h': 0}: 1
